# ADC multi 3rd
From an analog input, produce a digital output. We explore multiple architectures.

# Preamble

In [1]:
%load_ext autoreload
%autoreload 2
#%matplotlib widget
%matplotlib inline

In [2]:
import bqplot as bq
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['figure.max_open_warning'] = 0
import numpy as np
from sidecar import Sidecar
import time

Fetch our tools:

In [3]:
from lib.nn import Network, Layer, IdentityLayer, AffineLayer, MapLayer
from lib.nnbench import NNBench, NetMaker
from lib.nnvis import NNVis, ADCResponsePlot, NetResponsePlot

## A tool to plot the transfer function

In [4]:
%%script echo skipping (moved to nnvis.py)
class ADCResponsePlot():
    def __init__(self, margin=20,
                 min_aspect_ratio=0.5,
                 max_aspect_ratio=2,
                 title=None,
                 **kwargs):
        x = np.arange(-0.125, 1.125, 0.001)
        xs = bq.LinearScale()
        ys = bq.LinearScale()
        xax = bq.Axis(scale=xs, label='input')
        yax = bq.Axis(scale=ys, orientation='vertical', label='bits')
        line = bq.Lines(x=x, y=x, scales={'x': xs, 'y': ys})
        fig = self.fig = bq.Figure(marks=[line], axes=[xax, yax],
                                   min_aspect_ratio=min_aspect_ratio,
                                   max_aspect_ratio=max_aspect_ratio,
                                   title=title)
        dictfilt = lambda x, y: dict([ (i,x[i]) for i in x if i in set(y) ])
        fig.fig_margin = kwargs.get('fig_margin') \
            or dict(top=margin, bottom=margin, left=margin, right=margin)
        layout_defaults = {}
        layout_defaults.update(dictfilt(kwargs, ('height', 'width')))
        fig.layout = layout_defaults
    
    def __call__(self, fun):
        fig = self.fig
        line = fig.marks[0]
        #outs = np.array([fun(x) for x in line.x])
        outs = fun(line.x.reshape(-1,1))
        y = np.swapaxes(outs, 0, 1)
        line.y = y
        #return fig
        return self

skipping (moved to nnvis.py)


In [5]:
def plot_ADC(fun, **kwargs):
    return ADCResponsePlot(**kwargs)(fun).fig

In [6]:
%%script echo skipping (moved to nnvis.py)
class NetResponsePlot(ADCResponsePlot):
    def __init__(self, net, **kwargs):
        super().__init__(**kwargs)
        self.net = net

    def __call__(self, state_vector=None):
        net = self.net
        if state_vector:
            net.set_state_from_vector(state_vector)
        fig = self.fig
        line = fig.marks[0]
        outs = net(line.x.reshape(-1,1))
        y = np.swapaxes(outs, 0, 1)
        line.y = y
        return self

skipping (moved to nnvis.py)


## Reference implementation
Here's what we want to accomplish, but by network means:

### 3-bit linear binary output

In [7]:
def adc(input):
    m = max(0, min(7, int(8*input)))
    return np.array([(m>>2)&1, (m>>1)&1, m&1]) * 2 - 1

vadc = lambda v: np.array([adc(p) for p in v])
#plot_ADC(vadc)

### 3-bit linear Gray coded output

In [8]:
def gradc(input):
    m = max(0, min(7, int(8*input)))
    g = m ^ (m >> 1)
    return np.array([(g>>2)&1, (g>>1)&1, g&1]) * 2 - 1

vgradc = lambda v: np.array([gradc(p) for p in v])
#plot_ADC(vgradc)

___

# Networks
We create some variations, which we will train all at once in a race.

## Use `NetMaker` to make the networks from shorthand

In [9]:
#titles = '1x3t 1x8tx8tx3t 1x8sx8sx3t 1x8tx8tx3tx3t'.split()
titles = ['1x8tx8tx3t', '1x8tx8tx3tx3t'] * 2
nm = NetMaker()
nets = [nm(s) for s in titles]

## Paul's hand-crafted Siren

In [10]:
net = Network()
net.extend(AffineLayer(1,3))
net.layers[-1].set_state_from_vector(np.array([1,2,4,.51,.51,.51])*np.pi*2)
net.extend(MapLayer(np.sin, np.cos))
net.extend(AffineLayer(3,3))
net.layers[-1].set_state_from_vector(np.concatenate(((np.eye(3) * 1).ravel(), np.zeros(3))))
net.extend(MapLayer(np.tanh, lambda d: 1.0 - np.tanh(d)**2))
nets.append(net)
titles.append("Paul's")

Save interesting state vectors, starting with our starting point

In [11]:
state_vectors = [list(net.state_vector() for net in nets)]

In [12]:
starting_eta = 0.01
for net in nets:
    net.eta = starting_eta

# Graphs to the right

In [13]:
grid = widgets.GridspecLayout(3, 2, height='680px',
                      grid_gap='10px',
                      justify_content='center',
                      align_items='top')

#nrps = np.array([NetResponsePlot(net, height='220px', margin=30) \
#            for i in range(grid.n_rows * grid.n_columns)]).reshape(grid.n_rows, grid.n_columns)
nrps = np.array([NetResponsePlot(net, height='220px', margin=30, title=title) for net, title in zip(nets, titles)])

for i, nrp in enumerate(nrps):
    column = i % grid.n_columns
    row = i // grid.n_columns
    grid[row, column] = nrp.fig

batch_w = widgets.FloatText(value=-1.0, description='Batch:', max_width=6, disabled=True)
eta_w = widgets.FloatLogSlider(
    value=starting_eta,
    base=10,
    min=-4,
    max=0, # min exponent of base
    step=0.1, # exponent step
    description='eta'
)

def on_eta_change(change):
    eta = change['new']
    for net in nets:
        net.eta = eta

eta_w.observe(on_eta_change, names='value')

grid[-1,-1] = widgets.VBox((batch_w, eta_w))
        
with Sidecar(title='grid') as gside:
    display(grid)

In [14]:
_ = [nrp() for nrp in nrps]

In [15]:
nets[-1].eta

0.01

# Training
We will train our candidates in parallel, on the same training data, and watch their evolution.

## Training data

In [16]:
x = np.arange(0, 1, 1.0/(8*8)).reshape(-1,1) # 8 points in each output region
training_batch_cluster = [(x, vadc(x))]

In [17]:
#training_batch_cluster

## Training loop

In [18]:
batch = 0

In [19]:
for i in range(10):
    for net in nets:
        net.learn(training_batch_cluster)
    _ = [nrp() for nrp in nrps]
    batch += 1
    batch_w.value = batch
    time.sleep(0.5)

In [20]:
for i in range(100):
    for net in nets:
        net.learn(training_batch_cluster)
    batch += 1
    if i % 10 == 0:
        _ = [nrp() for nrp in nrps]
        batch_w.value = batch

In [ ]:
for i in range(100_000):
    for net in nets:
        net.learn(training_batch_cluster)
    batch += 1
    if i % 100 == 0:
        _ = [nrp() for nrp in nrps]
        batch_w.value = batch

In [ ]:
for i in range(100_000):
    for net in nets:
        net.learn(training_batch_cluster)
    batch += 1
    if i % 500 == 0:
        _ = [nrp() for nrp in nrps]
        batch_w.value = batch

In [ ]:
assert False, "stop here"

In [ ]:
for i in range(100_000):
    for j in range(min(int(2**(i/4)), 500)):
        for net in nets:
            net.learn(training_batch_cluster)
        batch += 1
    _ = [nrp() for nrp in nrps]
    batch_w.value = batch

In [ ]:
nrps[0]()

In [ ]:
nrps[1]()

In [ ]:
nrps[3].net.layers

In [ ]:
nets

In [ ]:
nrps

In [ ]:
[hash(nrp.fig) for nrp in nrps]

In [ ]:
nrps[3].net(np.array([0.5]))